### Get 16S sequence embedded into a model,may need to not use just a general language model.

In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch
import random
from tqdm import tqdm

In [2]:
def get_key(my_dict, val):
    keys = []
    for key, value in my_dict.items():
        if isinstance(value, list) and val in value:
            keys.append(key)
    return keys

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2') # can i use a biological dna model from hugging face? 

In [4]:
# Single list of sentences - Possible tens of thousands of sentences
sentences = [
    "TGCTTCTTGGGTGTTGAGCGTGACC",
    "GTGATCCCTTCGTAGCACCTGGGAA",
    "GAGCTCGAATAGGTTATCAGAGCAG",
    "CGACCGATGTCAAGAGCGCGCGCGT",
    "TCTTACCTACTTGACTCTAGTGCGG",
    "CCGACTGTTAGTCTCAGTTGTACTA",
    "TGCGAACGTGGACCTGGTGCGAAGC",
    "AGCTTGCGTGGATTCGTAGGGCTGG",
    "CGTCCCGTTGCATATCACGAGAGCG",
    "GGTTACACGTCTACTACTACATCCG"]


paraphrases = util.paraphrase_mining(model, sentences)

for paraphrase in paraphrases[0:5]:
    score, i, j = paraphrase
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))

TGCTTCTTGGGTGTTGAGCGTGACC 		 TGCGAACGTGGACCTGGTGCGAAGC 		 Score: 0.9140
CGTCCCGTTGCATATCACGAGAGCG 		 CGACCGATGTCAAGAGCGCGCGCGT 		 Score: 0.9005
CGTCCCGTTGCATATCACGAGAGCG 		 TGCGAACGTGGACCTGGTGCGAAGC 		 Score: 0.8369
TGCTTCTTGGGTGTTGAGCGTGACC 		 CGTCCCGTTGCATATCACGAGAGCG 		 Score: 0.8211
CGACCGATGTCAAGAGCGCGCGCGT 		 TGCGAACGTGGACCTGGTGCGAAGC 		 Score: 0.8138


In [5]:
from Bio import SeqIO
import pandas as pd

# Set the path to your FASTA file
fasta_file = "data/ready4train_seqs.fasta"

# Parse the sequences and headers from the FASTA file
records = SeqIO.parse(fasta_file, "fasta")

# Create a DataFrame from the sequences and headers
df = pd.DataFrame([(record.id, str(record.seq)) for record in records],
                  columns=["id", "sequence"])
df = df.set_index('id')
# Print the first 10 rows of the DataFrame
df.head()

,sequence
id,
GB_GCA_009371985.2,GAAGAGTTTGATCCTGGCTCAGGATGAACGCTAGCGGCAGGCCTAA...
RS_GCF_902705855.1,GGAGGTGATCCAGCCCCAGGTTCCCCTAGGGCTACCTTGTTACGAC...
RS_GCF_000426325.1,AGGAGGTGATCCAGCCGCACCTTCCGGTACGGCTACCTTGTTACGA...
RS_GCF_004343615.1,TGAGAGTTTGATCCTGGCTCAGAGCGAACGCTGGCGGCATGCTTAA...
RS_GCF_010727475.1,AGGAGGTGATCCAGCCGCACCTTCCGGTACGGCTACCTTGTTACGA...


In [6]:
taxonomy = pd.read_csv('data/train_taxonomy.tsv', sep='\t', index_col = 0)
taxonomy.head()

,domain,phylum,class,order,family,genus,species
Seq_ID,,,,,,,
RS_GCF_003697165.2,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia,Escherichia coli
RS_GCF_001027105.1,Bacteria,Firmicutes,Bacilli,Staphylococcales,Staphylococcaceae,Staphylococcus,Staphylococcus aureus
RS_GCF_000006945.2,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Salmonella,Salmonella enterica
RS_GCF_000742135.1,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Klebsiella,Klebsiella pneumoniae
RS_GCF_001457635.1,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus pneumoniae


In [7]:
taxonomy['sequence'] = df['sequence']

In [8]:
# Get rows with at least two occurrences of the value in the "genus" column
genus_counts = taxonomy['genus'].value_counts()
genus_multiple_occurrences = genus_counts[genus_counts >= 2].index.tolist()
df_subset = taxonomy[taxonomy['genus'].isin(genus_multiple_occurrences)]

# Split the DataFrame into two subsets, test and ref, with about 50% of each genus value
test = pd.DataFrame()
ref = pd.DataFrame()
for genus in genus_multiple_occurrences:
    genus_subset = df_subset[df_subset['genus'] == genus]
    n = len(genus_subset)
    indices = np.arange(n)
    np.random.shuffle(indices)
    split_idx = int(np.floor(n / 2))
    test_idx = indices[:split_idx]
    ref_idx = indices[split_idx:]
    test = pd.concat([test, genus_subset.iloc[test_idx]])
    ref = pd.concat([ref, genus_subset.iloc[ref_idx]])

# Print the resulting DataFrames
#print(test)
#print(ref)

In [9]:
q = 100

In [10]:
# Extract the relevant columns from the dataframes
test_subset = test[['genus', 'sequence']][0:q]
ref_subset = ref[['genus', 'sequence']][0:q]

# Set genus column as index for text and ref dataframes
test_subset.set_index('genus', inplace=True)
ref_subset.set_index('genus', inplace=True)

# Concatenate text and ref dataframes
combined_df = pd.concat([test_subset, ref_subset])

# Group sequences by genus
seq_dict = combined_df.groupby('genus')['sequence'].apply(list).to_dict()

In [11]:
# Initialize variables for evaluation metrics
total_accuracy = 0
count = 0

test_list = test.sequence[0:q].tolist()
ref_list = ref.sequence[0:q].tolist()

# Single list of sentences - Possible tens of thousands of sentences
sequences = test_list + ref_list
paraphrases = util.paraphrase_mining(model, sequences)  

# Group sublists by their second element (index 1)
grouped_dict = {}
for sublist in paraphrases:
    key = sublist[1]
    if key not in grouped_dict:
        grouped_dict[key] = []
    grouped_dict[key].append(sublist)

# Get the two sublists with the greatest value in element 0 for each unique value in element 1
result = []
for key in sorted(grouped_dict.keys()):
    sublists = grouped_dict[key]
    top_two_sublists = sorted(sublists, key=lambda x: x[0], reverse=True)[:1]# can change to get more or less sentence matching
    result.extend(top_two_sublists)

# Replace indices with corresponding sentences
for sublist in result:
    sublist[1] = sequences[sublist[1]]
    sublist[2] = sequences[sublist[2]]

for seq in tqdm(test_list):
    # Get all sublists where element 1 equals the match string
    closest_match = [sublist[2] for sublist in result if ((sublist[1] == seq))]
    correct_match = [key for key, values in seq_dict.items() if seq in values]
    closest_match = get_key(seq_dict, closest_match[0])
    accuracy = int(any(item in closest_match for item in correct_match))
    
    # Update evaluation metrics
    total_accuracy += accuracy
    count += 1
    
    #print(f"Input taxa: {get_key(seq_dict, seq)}, closest match: {closest_match}, correct match: {correct_match}")

# Compute average evaluation metrics
avg_accuracy = total_accuracy / count

print(f"Average Accuracy: {avg_accuracy}")
%time

100%|██████████| 100/100 [00:00<00:00, 49730.90it/s]

Average Accuracy: 1.0
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [12]:
# Initialize variables for evaluation metrics
total_accuracy = 0
count = 0

test_list = test.sequence[0:q].tolist()
ref_list = ref.sequence[0:q].tolist()

# Single list of sentences - Possible tens of thousands of sentences
sequences = test_list + ref_list
paraphrases = util.paraphrase_mining(model, sequences)  

# Group sublists by their second element (index 1)
grouped_dict = {}
for sublist in paraphrases:
    key = sublist[1]
    if key not in grouped_dict:
        grouped_dict[key] = []
    grouped_dict[key].append(sublist)

# Get the two sublists with the greatest value in element 0 for each unique value in element 1
result = []
for key in sorted(grouped_dict.keys()):
    sublists = grouped_dict[key]
    top_two_sublists = sorted(sublists, key=lambda x: x[0], reverse=True)[:1]# can change to get more or less sentence matching
    result.extend(top_two_sublists)

# Replace indices with corresponding sentences
for sublist in result:
    sublist[1] = sequences[sublist[1]]
    sublist[2] = sequences[sublist[2]]

# Create a dictionary that maps each sequence to its closest match
closest_match_dict = {}
for seq in test_list:
    # Get all sublists where element 1 equals the match string
    closest_match = [sublist[2] for sublist in result if ((sublist[1] == seq))]
    closest_match_dict[seq] = get_key(seq_dict, closest_match[0])

# Compute accuracy for each test sequence
for seq, closest_match in tqdm(closest_match_dict.items()):
    correct_match = [key for key, values in seq_dict.items() if seq in values]
    accuracy = int(any(item in closest_match for item in correct_match))
    
    # Update evaluation metrics
    total_accuracy += accuracy
    count += 1
    
    #print(f"Input taxa: {get_key(seq_dict, seq)}, closest match: {closest_match}, correct match: {correct_match}")

# Compute average evaluation metrics
avg_accuracy = total_accuracy / count

print(f"Average Accuracy: {avg_accuracy}")
%time

100%|██████████| 100/100 [00:00<00:00, 209296.61it/s]

Average Accuracy: 1.0
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


### can also try below with DNA sequence model

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# Import the tokenizer and the model
#tokenizer = AutoTokenizer.from_pretrained("InstaDeepAI/nucleotide-transformer-2.5b-multi-species")
#model = AutoModelForMaskedLM.from_pretrained("InstaDeepAI/nucleotide-transformer-2.5b-multi-species")

# Create a dummy dna sequence and tokenize it
sequences = test_list + ref_list
tokens_ids = tokenizer.batch_encode_plus(sequences, return_tensors="pt", padding=True, truncation=True)["input_ids"]

# Compute the embeddings
attention_mask = tokens_ids != tokenizer.pad_token_id
torch_outs = model(
    tokens_ids,
    attention_mask=attention_mask,
    encoder_attention_mask=attention_mask,
    output_hidden_states=True
)

# Compute sequences embeddings
embeddings = torch_outs['hidden_states'][-1].detach().numpy()
print(f"Embeddings shape: {embeddings.shape}")
print(f"Embeddings per token: {embeddings}")

# Compute mean embeddings per sequence
mean_sequence_embeddings = torch.sum(attention_mask.unsqueeze(-1)*embeddings, axis=-2)/torch.sum(attention_mask, axis=-1)
print(f"Mean sequence embeddings: {mean_sequence_embeddings}")

### need to get data andfind method to compare label to true or false label, or I can try to get database from green genes and then compare the ability of this method to the classification of greengenes using a real data set

In [ ]:
# Create a dummy dna sequence and tokenize it
sequences = 
tokens_ids = tokenizer.batch_encode_plus(sequences, return_tensors="pt")["input_ids"]

# Compute the embeddings
attention_mask = tokens_ids != tokenizer.pad_token_id
torch_outs = model(
    tokens_ids,
    attention_mask=attention_mask,
    encoder_attention_mask=attention_mask,
    output_hidden_states=True
)

# Compute sequences embeddings
embeddings = torch_outs['hidden_states'][-1].detach().numpy()
print(f"Embeddings shape: {embeddings.shape}")
print(f"Embeddings per token: {embeddings}")

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Import the tokenizer and the model
#tokenizer = AutoTokenizer.from_pretrained("InstaDeepAI/nucleotide-transformer-2.5b-multi-species")
#model = AutoModelForMaskedLM.from_pretrained("InstaDeepAI/nucleotide-transformer-2.5b-multi-species")

# Define a dictionary of reference sequences and their embeddings
ref_sequences = {
    "ref_seq_1": "ATTCTG" * 9,
    "ref_seq_2": "GTAGTC" * 9,
    "ref_seq_3": "CTCGAT" * 9
}
ref_embeddings = {}

# Compute the embeddings for the reference sequences
for name, seq in ref_sequences.items():
    tokens_ids = tokenizer.encode(seq, return_tensors="pt")
    attention_mask = tokens_ids != tokenizer.pad_token_id
    torch_outs = model(tokens_ids, attention_mask=attention_mask, encoder_attention_mask=attention_mask, output_hidden_states=True)
    embeddings = torch_outs['hidden_states'][-1].detach().numpy()
    ref_embeddings[name] = np.mean(embeddings, axis=1)

# Define a list of input sequences
input_sequences = [
    "ATTCTG" * 9,
    "AGAGTT" * 9,
    "CTCGAT" * 9
]

# Encode the input sequences and compute their embeddings
input_tokens = tokenizer.batch_encode_plus(input_sequences, return_tensors="pt", padding=True)
input_embeddings = model(input_tokens['input_ids'], attention_mask=input_tokens['attention_mask'], encoder_attention_mask=input_tokens['attention_mask'], output_hidden_states=True)['hidden_states'][-1].detach().numpy()

# Compute cosine similarity between input embeddings and reference embeddings
cos_sim = cosine_similarity(input_embeddings, [ref_embeddings[name] for name in ref_sequences])

# Find the closest reference sequence for each input sequence
closest_seqs = [max(ref_sequences, key=lambda name: cos_sim[i][list(ref_sequences).index(name)]) for i in range(cos_sim.shape[0])]

# Print the closest reference sequence for each input sequence
for i, seq in enumerate(input_sequences):
    print(f"Input sequence {i+1}: {seq}")
    print(f"Closest reference sequence: {closest_seqs[i]}\n")


In [ ]:
# Load reference sequences and compute embeddings
ref_sequences = ["ATCGATCG", "CGTAGCTA", "TTGACCTA"]
ref_embeddings = {}
for seq in ref_sequences:
    tokens_ids = tokenizer.encode(seq, return_tensors="pt")
    torch_outs = model(tokens_ids)
    ref_embeddings[seq] = torch_outs.hidden_states[-1][0]

# Create a dummy input sequence and compute its embedding
input_sequence = "CGTAGGCA"
input_ids = tokenizer.encode(input_sequence, return_tensors="pt")
with torch.no_grad():
    model_output = model(input_ids)
    input_embedding = model_output.hidden_states[-1][0]

# Compute the cosine similarity between the input embedding and the reference sequence embeddings
cos_sim = cosine_similarity(input_embedding.reshape(1, -1), [ref_embeddings[name] for name in ref_sequences])
print(cos_sim)